In [13]:
### ========== Version 1 Azure SDK
from azure.identity import InteractiveBrowserCredential
from azureml.core import Workspace, Experiment, Dataset, Datastore
from azureml.core.authentication import InteractiveLoginAuthentication

### ========== Version 2 Azure SDK
from azure.ai.ml import command, MLClient, Input, Output, dsl
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml.entities import Environment, Model, ManagedOnlineEndpoint, PipelineJob

In [14]:
from azure.src.cloud_evaluating import ModelEval
from keras.models import load_model
import os

subscription_id = "0a94de80-6d3b-49f2-b3e9-ec5818862801"
resource_group = "buas-y2"
workspace_name = "CV5"

config_path = "config.json"
compute_target = "adsai1"
env_name = "cv5-env-simona"
ver = 6
curated_env_name = "AzureML-tensorflow-2.16-cuda11@latest"
auth = InteractiveLoginAuthentication()
credentials = InteractiveBrowserCredential()
workspace = Workspace(subscription_id=subscription_id, resource_group=resource_group, workspace_name=workspace_name,
                      auth=auth)
ml_client = MLClient(
    credential=credentials,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name)
env = ml_client.environments.get(env_name, ver)

07:30:23 - Initializing with Entra authority: https://login.microsoftonline.com/0a33589b-0036-4fe8-a829-3ed0926af886
07:30:23 - Initializing with Entra authority: https://login.microsoftonline.com/0a33589b-0036-4fe8-a829-3ed0926af886
07:30:23 - Initializing with Entra authority: https://login.microsoftonline.com/0a33589b-0036-4fe8-a829-3ed0926af886
07:30:24 - Initializing with Entra authority: https://login.microsoftonline.com/organizations
07:30:27 - InteractiveBrowserCredential.get_token succeeded


In [15]:
train_component = ml_client.components.get(name="training")

07:30:31 - InteractiveBrowserCredential.get_token succeeded


In [18]:
def train_predict_pipeline(train_images_uri: str, train_masks_uri: str) -> None:
    training_step = train_component(images=train_images_uri, masks=train_masks_uri)
    training_step.outputs.model


In [22]:
train_images_path = Input(path="azureml://subscriptions/0a94de80-6d3b-49f2-b3e9-ec5818862801/resourcegroups/buas-y2/workspaces/CV5/datastores/workspaceblobstore/paths/sample_patches/sample_8/images")
train_masks_path = Input(path="azureml://subscriptions/0a94de80-6d3b-49f2-b3e9-ec5818862801/resourcegroups/buas-y2/workspaces/CV5/datastores/workspaceblobstore/paths/sample_patches/sample_8/masks")
test_path = Input(path="azureml://subscriptions/0a94de80-6d3b-49f2-b3e9-ec5818862801/resourcegroups/buas-y2/workspaces/CV5/datastores/workspaceblobstore/paths/test_set_raw_images")
## these are set to the train just to test the pipeline
val_images_path = Input(path="azureml://subscriptions/0a94de80-6d3b-49f2-b3e9-ec5818862801/resourcegroups/buas-y2/workspaces/CV5/datastores/workspaceblobstore/paths/sample_patches/sample_8/images")
val_masks_path = Input(path="azureml://subscriptions/0a94de80-6d3b-49f2-b3e9-ec5818862801/resourcegroups/buas-y2/workspaces/CV5/datastores/workspaceblobstore/paths/sample_patches/sample_8/masks")

In [24]:
training_step = train_component(images=train_images_path, masks=train_masks_path).ouputs.model
training_step.outputs.model

{}

In [9]:
model_uri = "azureml:azureml_245f9409-7b80-4990-ac93-d4d354290c2f_output_data_model:1"

In [10]:
modelling = ModelEval()

In [11]:
def load_model_keras(model_uri):
    #model = load_model(os.path.join(model_uri, f"model.keras"), custom_objects={'f1': modelling.f1, 'iou': modelling.iou})
    return load_model(model_uri, custom_objects={'f1': modelling.f1, 'iou': modelling.iou})

In [12]:
model = load_model_keras(model_uri)

ValueError: File format not supported: filepath=azureml:azureml_f69d1587-8481-4796-ad61-79e81361ad34_output_data_model:1. Keras 3 only supports V3 `.keras` files and legacy H5 format files (`.h5` extension). Note that the legacy SavedModel format is not supported by `load_model()` in Keras 3. In order to reload a TensorFlow SavedModel as an inference-only layer in Keras 3, use `keras.layers.TFSMLayer(azureml:azureml_f69d1587-8481-4796-ad61-79e81361ad34_output_data_model:1, call_endpoint='serving_default')` (note that your `call_endpoint` might have a different name).

In [ ]:
  def eval(self, image_folder, mask_folder, model_folder):
    model = self.load_model_keras(model_folder)
    validation_generator, test_generator, validation_image_generator, test_image_generator = self.data_generator(image_folder, mask_folder)
    evaluation = model.evaluate(test_generator, steps=validation_image_generator.samples)


    eval_loss = evaluation[0]
    eval_acc = evaluation[1]
    eval_f1 = evaluation[2]
    eval_iou = evaluation[3]

    mlflow.log_metric("loss", evaluation[0])
    mlflow.log_metric("accuracy", evaluation[1])
    mlflow.log_metric("f1", evaluation[2])
    mlflow.log_metric("iou", evaluation[3])
